In [1]:
import h5py
import pyccl
import scipy
import numpy
import json

In [2]:
# Load in Data

PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'
DATA_PATH = PATH + 'DATA/'
CODE_PATH = PATH + 'CODE/'
PLOT_PATH = PATH + 'PLOT/'

BIN_SIZE = 4
GRID_SIZE = 72
LATENT_SIZE = 32
DATA_SIZE = 100000
COLOR = ['blue', 'green', 'orange', 'red']

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as FILE:
    
    Z_GRID = numpy.array(FILE['realizations/z'][:GRID_SIZE + 1], dtype = 'double')
    GRID = numpy.array(FILE['realizations/pdfs'][:, :BIN_SIZE, :GRID_SIZE + 1], dtype = 'double')

In [3]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as FILE:
    COSMO = json.load(FILE)
    
COSMO_CCL = pyccl.cosmology.Cosmology(
    h=COSMO['H'],
    w0=COSMO['W0'],
    wa=COSMO['WA'],
    n_s=COSMO['NS'],
    m_nu=COSMO['MNU'],
    Neff=COSMO['NEFF'],
    T_CMB=COSMO['TCMB'],
    mass_split='single',
    sigma8=COSMO['SIGMA8'],
    Omega_k=COSMO['OMEGAK'],
    Omega_c=COSMO['OMEGAC'],
    Omega_b=COSMO['OMEGAB'],
    mg_parametrization=None,
    matter_power_spectrum='halofit',
    transfer_function='boltzmann_camb',
    extra_parameters={
        'camb': {'kmax': 10000, 'lmax': 10000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# PMM, matter-matter-power-spectra

PSI_GRID = numpy.mean(GRID, axis = 0)
SIGMA_GRID = numpy.std(GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + Z_GRID))
CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + Z_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 100
ELL_DATA = numpy.logspace(1, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where = CHI_MESH > 0))

PMM_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):

    PMM_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [5]:
#numerical integral

def integralI5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    def formulan(chi):
        
        result = (chi2-chi) / (chi*(chi2 - chi1))

        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))

        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)

        return result

    def formula0(chi):
        
        result = (chi2-chi) / (chi*(chi2 - chi1))

        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))

        result = result * (((chi/chi2)**3)*power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)

        return result

    if chi1 == 0:
        
        integral, error = scipy.integrate.quad_vec(f = formula0, a = chi1, b = chi2)
        
    else:
        
        integral, error = scipy.integrate.quad_vec(f = formulan, a = chi1, b = chi2)

    return integral

In [6]:
#coefficient equation

def coefficientJ5(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    y = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)

    if a == 1:

        formula = (7 * b * (7 + 5 * b) - 2 * b * (9 + 7 * b) * z + 14 * (1 + b) * ( - 3 + z) * numpy.log(1 + b)) / (840 * b)
        
    else:

        formula = - ((1 / (720 * a**4 * b)) * (a * b * (10 * a * (6 * a * ( - 12 + 5 * a**2 + 6 * a * (2 + b)) + 60 * y - a * (66 + 28 * a + 17 * a**2 + 18 * (2 + a) * b) * y) + 
( - 10 * a * ( - 60 + a * (66 + 28 * a + 17 * a**2 + 18 * (2 + a) * b)) + 3 * ( - 180 + a * (30 * (7 + 4 * b) + a * (90 + 60 * b + a * (55 + 39 * a + 40 * b)))) * y) * z) + 
60 * (b * (6 * a**3 * (3 + b) + 10 * a * y - 9 * y * z + a * (10 + 15 * y + 6 * b * y) * z - 2 * a**2 * (6 + 8 * y + 3 * b * y + 8 * z + 3 * b * z) + a**5 * ( - 6 + 4 * y + 4 * z - 3 * y * z) + 
a**4 * (6 - 4 * y - 4 * z + 3 * y * z)) * numpy.log(1 - a) + a**5 * (1 + b) * (6 - 4 * z + y * ( - 4 + 3 * z)) * numpy.log(1 + b))))
        
    coefficient = chi2* power2 * (1 + redshift2) * formula
    
    return coefficient

In [7]:
#Case1 test: For chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.30684999374 1497.8183354014277 1534.1133113296617 49919.45246058225 48958.62248873209 0.36 0.37


In [8]:
#Case1 test: For chi1 > 0

INTEGRAL = integralI5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(505.23963699385405, 505.2396371252358, 2.600384352291485e-10)

In [9]:
#Case2 test: For chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.42674493673601 88.64121385461215 0.0 2114.2567965927065 0.0 0.01


In [10]:
#Case2 test: For chi1 = 0

INTEGRAL = integralI5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ5(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(2885.122308813219, 2885.12230881322, 2.220446049250313e-16)